In [1]:
import math
from sklearn import neighbors
import os
from time import sleep
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
import pygame
from gtts import gTTS
from tempfile import TemporaryFile
import time
import cv2
import threading

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
#global video_capture 
#video_capture = cv2.VideoCapture(1)
global frame, is_exit, video_capture

def capture():
    while not is_exit:
        # Grab a single frame of video
        ret, frame = video_capture.read()

def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    
    X = []
    y = []

    # Recorre a través de cada persona en el conjunto de entrenamiento
    for class_dir in os.listdir(train_dir):
        
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        # Recorre cada imagen de entrenamiento para la persona actual
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)
                     
            if len(face_bounding_boxes) != 1:
                
                # Filtro de muchas caras
                if verbose:
                    print("Imagen {} no es recomendable para el entrenamiento: {}".format(img_path, "No se encontraron caras" if len(face_bounding_boxes) < 1 else "Muchas caras encontradas"))
            else:
                # Codifica imagen actual del conjunto de entrenamiento
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)
                

    # Calcula numero de vecinos
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Numero de vecinos encontrados:", n_neighbors)

    # Crea y entrena el KNN
    
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')   
    knn_clf.fit(X, y)


    # Guarda el kernel generado
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf


def predict(X_img, knn_clf=None, model_path=None, distance_threshold=0.6):
   
    #Validacion de extension 
    #if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
       # raise Exception("Formato de imagen no admitido: {}".format(X_img_path))

    #validacion de ruta de clasificador ya entrenado
    if knn_clf is None and model_path is None:
        raise Exception("Debe proporcionar el archivo del clasificador o la ruta ")

    # carga un modelo entrenado
    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    # carga archivo de imagen 
    #X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    # Devuelve cero si no hay caras
    if len(X_face_locations) == 0:
        return []

    # Encontrar codificaciones para caras en el image.
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    # Utilizamos el knn en nuestro conjunto definido para la cara de prueba
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    #print(closest_distances)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Predecir clases y eliminar clasificaciones que no están dentro del umbral
    return [(pred, loc) if rec else ("Desconocido", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


def show_prediction_labels_on_image(img_path, predictions):
   
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        # Dibuja un cuadro
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
       
        name = name.encode("UTF-8")

        # Dibuja una etiqueta con un nombre debajo de la cara
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    # Elimina la biblioteca de dibujos de memoria RAM
    del draw
    # Muestra la imagen resultante
    pil_image.show()

if __name__ == "__main__":

    video_capture = cv2.VideoCapture(0)
    #print(video_capture)
    #is_exit = False
    #capture_thread = threading.Thread(target=capture, name='captureThread', daemon=True)
    #capture_thread.start()

    Known_face_personalHitss=[
        "Alan",
        "Javier Bardem",
        "Jeffrey Dean Morgan",
        "Mark Walberg",
        "Matt Daemon",
        "Obama",
        "Ollanta",
        "Toledo",
        "Vizcarra"
    ]

    Known_face_bandera=[
        0,
        0,
        0,
        0,
        0,
        0,
        0,
        0
    ]
    
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    print("Entrenando Clasificador KNN")
    classifier = train("ClasificadorKNN/train", model_save_path="trained_knn_model.clf", n_neighbors=2)
    print("Entrenamiento Completo!")
    bandera=0

    while True:
        
        ret, frame = video_capture.read()

    
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    
        rgb_small_frame = small_frame[:, :, ::-1]

        X_img = rgb_small_frame
        #X_img = face_recognition.load_image_file("descarga.jpeg")
        #full_file_path = os.path.join("ClasificadorKNN/test", image_file)
        if process_this_frame:

            face_locations = face_recognition.face_locations(X_img)#,number_of_times_upsample=2)
            face_encodings = face_recognition.face_encodings(X_img)
            #print("Buscando caras en {}".format(image_file))
            #print(face_locations)
            predictions = []
            print(predictions)
            #predictions = predict(X_img, model_path="trained_knn_model.clf",distance_threshold=0.44)
            
            if face_locations:
                predictions = predict(X_img, model_path="trained_knn_model.clf",distance_threshold=0.44)
                print(predictions)
                if predictions:
                    #print("BUENOS DIAS AMIGOS")
                    #print(predictions[0][0])
                    #print(type(predictions[0][0]))
                    bandera=0
                    if bandera == 0:
                        if predictions[0][0]!='Desconocido':
                            tts = gTTS('Buenos Dias '+str(predictions[0][0]),'es-es')
                            tts.save('BD.mp3')
                            pygame.mixer.init()
                            pygame.mixer.music.load("BD.mp3") 
                            #pygame.mixer.music.play()
                            pygame.mixer.music.play()
                            #bandera = 1
                            print("CONTROL")
                        else:
                            tts = gTTS('Buenos Dias Por favor identifiquese con la recepcionista','es-es')
                            tts.save('BD.mp3')
                            pygame.mixer.init()
                            pygame.mixer.music.load("BD.mp3") 
                            #pygame.mixer.music.play()
                            pygame.mixer.music.play()
                            #bandera = 1
                            print("CONTROL")
                #print("Buenos Dias "+predictions[0][0])
        process_this_frame= not process_this_frame

            #else:
                #predictions = predict(X_img, model_path="trained_knn_model.clf",distance_threshold=0.44)
                #print(predictions)
                #print("AQUI NO HAY NADIE")
                #print(face_locations)
            #print(full_file_path)
            
            #for name, (top, right, bottom, left) in predictions:
                #print("- Encontrado  {} en ({}, {})".format(name, left, top))

            #show_prediction_labels_on_image(os.path.join("ClasificadorKNN/test", X_img), predictions)
        for (top, right, bottom, left), name in zip(face_locations, face_names):
        
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        #cv2.imshow('Video', frame)

    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'face_recognition'